In [1]:
import OpenDartReader # 모듈 불러오기
my_api = "" # API KEY 지정
dart = OpenDartReader(my_api) # OpenDartReader 객체 생성하기

In [3]:
import pandas as pd

stock_list = pd.read_csv("../../데이터/종목정보.txt", encoding="euc-kr", sep="\t", usecols=["Name", "Symbol"], dtype=str)

# pandas에서 특정 컬럼에 대해 타입을 정해줄 경우 컬럼명을 key로, 타입을 value로 하는 dict 형태로 dtype 파라미터에 지정해줘야 함.

In [4]:
display(stock_list.head())
print(len(stock_list))

,Symbol,Name
0,060310,3S
1,095570,AJ네트웍스
2,006840,AK홀딩스
3,054620,APS홀딩스
4,265520,AP시스템


2352


In [2]:
# finstate 메서드 반환값에서 파싱할 부분을 찾아본다. 자산총계, 부채총계, 자본총계, 매출액, 영업이익, 당기순이익

_report = dart.finstate("005380", 2020)
display(_report[['fs_nm', 'account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']])

,fs_nm,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,연결재무제표,유동자산,"83,686,357,000,000","76,082,873,000,000","73,008,101,000,000"
1,연결재무제표,비유동자산,"125,657,855,000,000","118,429,347,000,000","107,647,651,000,000"
2,연결재무제표,자산총계,"209,344,212,000,000","194,512,220,000,000","180,655,752,000,000"
3,연결재무제표,유동부채,"59,459,528,000,000","53,314,096,000,000","49,438,414,000,000"
4,연결재무제표,비유동부채,"73,543,714,000,000","64,832,370,000,000","57,321,328,000,000"
5,연결재무제표,부채총계,"133,003,242,000,000","118,146,466,000,000","106,759,742,000,000"
6,연결재무제표,자본금,"1,488,993,000,000","1,488,993,000,000","1,488,993,000,000"
7,연결재무제표,이익잉여금,"68,911,800,000,000","68,249,633,000,000","66,490,082,000,000"
8,연결재무제표,자본총계,"76,340,970,000,000","76,365,754,000,000","73,896,010,000,000"
9,연결재무제표,매출액,"103,997,601,000,000","105,746,422,000,000","96,812,609,000,000"


In [30]:
sum(_report["fs_nm"] == "연결재무제표")

12

In [3]:
_report.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
       'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
       'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
       'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord', 'currency'],
      dtype='object')

In [4]:
_report.loc[_report['account_nm'] == '자산총계', ['thstrm_amount']] #.iloc[0].values


,thstrm_amount
2,"209,344,212,000,000"
14,"78,252,289,000,000"


종목명과 사업연도, 재무지표 목록을 입력하면 관련 사업보고서에서 당기, 전기, 전전기의 재무지표를 추출하는 함수를 작성.   
이 함수의 출력은 세 개의 행으로 구성되는 df으로, 각 행은 순서대로 당기, 전기, 전전기의 기업, 연도, 재무지표를 나타냄

In [5]:
# 입력: stock_name, year, [자산총계, 부채총계, 자본총계, 매출액, 영업이익, 당기순이익]

def find_fins(stock_code, stock_name, year, account_nm):
    _report = dart.finstate(stock_code, year)
    for code, name in stock_list[['Name', 'Symbol']].values:
        for year in [2015, 2018, 2020]:
            cur_val = _report.loc[_report['account_nm'] == account_nm, ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].iloc[0].values[0]
            pre_val = _report.loc[_report['account_nm'] == account_nm, ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].iloc[0].values[1]
            spre_val = _report.loc[_report['account_nm'] == account_nm, ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].iloc[0].values[2]
            


In [ ]:
import numpy as np
import pandas as pd

def find_fins(stock_code, stock_name, year, ind_list):
    _report = dart.finstate(stock_code, year)

    if _report is None:
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1 ] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2 ] + [np.nan] * len(ind_list))

    else:  
        _report = _report.loc[_report['account_nm'].isin(ind_list)]
        if sum(_report['fs_nm'] == '연결재무제표') > 0:
            _report = _report.loc[_report['fs_nm'] == '연결재무제표']
        else:
            _report = _report.loc[_report['fs_nm'] == '재무제표']
        
        data = []
        for y, c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, year]
            for ind in ind_list:
                if sum(_report['account_nm'] == ind) > 0:
                    value = _report.loc[_report['account_nm'] == ind, c].iloc[0]
                else:
                    value = np.nan
                record += value
            data.append(record)

In [29]:
stock_list[['Name', 'Symbol']].values

array([['3S', '060310'],
       ['AJ네트웍스', '095570'],
       ['AK홀딩스', '006840'],
       ...,
       ['흥아해운', '003280'],
       ['희림', '037440'],
       ['힘스', '238490']], dtype=object)

데이터  파싱 함수 작성

In [49]:
import numpy as np
import pandas as pd

def find_fins_ind_list(stock_code, stock_name, year, ind_list):
    try: # 데이터 가져오기
        report = None
        report = dart.finstate(stock_code, year)
    except:
        pass

    if report is None: # 리포트가 없다면(참고: 리포트가 없으면 None을 반환함)
        # 리포트가 없으면 당기, 전기, 전전기 값을 모두 제거
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))

    else: 
        report = report[report["account_nm"].isin(ind_list)] # 관련 지표로 필터링
        if sum(report["fs_nm"] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표를 사용
            report = report.loc[report["fs_nm"] == "연결재무제표"]
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report["fs_nm"] == "재무제표"]
        data = []
        for y, c in zip([year, year-1, year-2], ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan
                    
                record.append(value)
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [50]:
ind_list = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익']

In [ ]:
import time

data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정
for idx, code in enumerate(stock_list[['Code']].values):
    print(idx + 1, '/', len(stock_list))
    for stock_name in stock_list[['Name']].values:    
        for year in [2015, 2018, 2020]:
            try: # 재무제표 데이터가 잘 불러와지지 않는 경우가 있어, try - except으로 넘김
                result = find_fins_ind_list(code, stock_name, year, ind_list) # 재무지표 데이터 
            except:
                pass
            data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
            time.sleep(0.5)

In [56]:
import numpy as np
import pandas as pd

def find_financial_indicators(stock_code, stock_name, year, ind_list):
    report = dart.finstate(stock_code, year) # 데이터 가져오기
    if report is None:
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))
        return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)
    else:
        report = report[report['account_nm'].isin(ind_list)]
        if sum(report['fs_nm'] == '연결재무제표') > 0:
            report = report.loc[report['fs_nm'] == '연결재무제표']
        else:
            report = report.loc[report['fs_nm'] == '재무제표']

        data = []
        for y, c in zip([year, year-1, year-2], ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan

                record.append(value)
                
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [55]:
import numpy as np
import pandas as pd

def find_financial_indicators(stock_code, stock_name, year, ind_list):
    report = dart.finstate(stock_code, year) # 데이터 가져오기
    try:
        report = report[report['account_nm'].isin(ind_list)]
        if sum(report['fs_nm'] == '연결재무제표') > 0:
            report = report.loc[report['fs_nm'] == '연결재무제표']
        else:
            report = report.loc[report['fs_nm'] == '재무제표']
    except:
        pass
    if report is None:
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data = [[stock_name, year - 1] + [np.nan] * len(ind_list)]
        data = [[stock_name, year - 2] + [np.nan] * len(ind_list)]
        return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)
    
    data = []
    for y, c in zip([year, year-1, year-2], ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
        record = [stock_name, y]
        for ind in ind_list:
            # account_nm이 indic인 행의 c 컬럼 값을 가져옴
            if sum(report["account_nm"] == ind) > 0:
                value = report.loc[report["account_nm"] == ind, c].iloc[0]
            else:
                value = np.nan
            record.append(value)
                
        data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [ ]:
ind_list = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
display(find_financial_indicators("005930", "삼성전자", 2020, ind_list))

In [57]:
import time
data = pd.DataFrame()

for idx, stock_name in enumerate(stock_list):
    print(idx +1, '/', len(stock_list))
    for year in [2015, 2018, 2020]:
        try:
            result = find_financial_indicators(stock_name, year, ind_list)
        except:
            pass
        data = pd.concat([data, result], axis=0, ignore_index=True)
        time.sleep(0.5)

1 / 2481
2 / 2481


In [44]:
stock_list[['Code', 'Name']].values

array([['005930', '삼성전자'],
       ['373220', 'LG에너지솔루션'],
       ['000660', 'SK하이닉스'],
       ...,
       ['001527', '동양2우B'],
       ['002787', '진흥기업2우B'],
       ['000547', '흥국화재2우B']], dtype=object)

In [64]:
def fs_finder(stock_code, stock_name, year, ind_list):
    try: 
        report = dart.finstate(stock_code, year) # 데이터 가져오기
    except:
        report is None
    if report is None:
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))
        return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)
    else:
        report = report[report['account_nm'].isin(ind_list)]
        if sum(report['fs_nm'] == '연결재무제표') > 0:
            report = report.loc[report['fs_nm'] == '연결재무제표']
        else:
            report = report.loc[report['fs_nm'] == '재무제표']

        data = []
        for y, c in zip([year, year-1, year-2], ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan

                record.append(value)
                
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [67]:
import time

data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정
for code, name in stock_list[['Code', 'Name']].values:
    print(name)
    for year in [2015, 2018, 2020]:
        try: # 재무제표 데이터가 잘 불러와지지 않는 경우가 있어, try - except으로 넘김
            result = fs_finder(code, name, year, ind_list) # 재무지표 데이터 
        except:
            pass
        data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
        time.sleep(0.5)

삼성전자
LG에너지솔루션
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

SK하이닉스
삼성바이오로직스
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

LG화학
삼성SDI
현대차
기아
POSCO홀딩스
NAVER
포스코퓨처엠
에코프로비엠
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

카카오
셀트리온
현대모비스
삼성물산
KB금융
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

신한지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

LG전자
SK이노베이션
에코프로
LG
삼성생명
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

하나금융지주
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

SK
한국전력
KT&G
두산에너빌리티
카카오뱅크
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

{'status': '013',

데이터 정제

In [68]:
# data에서 중복된 행을 제거하고 '기업'과 '연도' 컬럼을 기준으로 정렬

data.drop_duplicates(inplace=True)
data.sort_values(by=['기업', '연도'], inplace=True)

In [69]:
display(data.head())

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
13097,3S,2013,"67,660,583,293","19,883,875,822","47,776,707,471","32,236,446,153","1,859,291,010","701,146,938"
13096,3S,2014,"71,312,012,159","28,319,186,484","42,992,825,675","28,862,985,297","-2,540,739,501","-4,760,633,476"
13095,3S,2015,"67,359,971,696","29,420,163,240","37,939,808,456","23,210,541,648","-4,157,989,321","-5,599,032,927"
13100,3S,2016,"57,746,389,196","23,142,629,825","34,603,759,371","23,742,421,655","1,176,535,923","-3,403,437,816"
13099,3S,2017,"53,706,903,697","24,527,236,916","29,179,666,781","26,486,287,918","-2,421,224,869","-8,803,810,083"


In [70]:
# 숫자로 모두 변환

def str_to_float(value):
    if type(value) == float: # nan의 자료형은 float임
        return value
    elif value == "-": # -로 되어 있으면 0으로 변환
        return 0
    else:
        return float(value.replace(",",""))
    
for ind in ind_list:
    data[ind] = data[ind].apply(str_to_float)

In [71]:
display(data.head())

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
13097,3S,2013,6.766058e+10,1.988388e+10,4.777671e+10,3.223645e+10,1.859291e+09,7.011469e+08
13096,3S,2014,7.131201e+10,2.831919e+10,4.299283e+10,2.886299e+10,-2.540740e+09,-4.760633e+09
13095,3S,2015,6.735997e+10,2.942016e+10,3.793981e+10,2.321054e+10,-4.157989e+09,-5.599033e+09
13100,3S,2016,5.774639e+10,2.314263e+10,3.460376e+10,2.374242e+10,1.176536e+09,-3.403438e+09
13099,3S,2017,5.370690e+10,2.452724e+10,2.917967e+10,2.648629e+10,-2.421225e+09,-8.803810e+09


In [72]:
data['부채비율'] = data['부채총계'] / data['자본총계'] * 100
display(data['부채비율'].head())

13097    41.618347
13096    65.869563
13095    77.544311
13100    66.878947
13099    84.055918
Name: 부채비율, dtype: float64

In [73]:
data['매출액증가율'] = (data['매출액'].diff() / data['매출액'].shift(1)) * 100
data.loc[data['연도'] == 2013, '매출액증가율'] = np.nan

In [74]:
data['영업이익증가율'] = (data['영업이익'].diff() / data['영업이익'].shift(1)) * 100
data.loc[data['연도'] == 2013, '영업이익증가율'] = np.nan

data['당기순이익증가율'] = (data['당기순이익'].diff() / data['당기순이익'].shift(1)) * 100
data.loc[data['연도'] == 2013, '당기순이익증가율'] = np.nan

In [75]:
data = data.replace({np.inf:np.nan, -np.inf: np.nan})

In [76]:
# 상태를 나타내는 함수 정의
def add_state(data, col):
    data[col + "_상태"] = np.nan # 상태를 결측으로 초기화
    value = data[col].values
    cur_value = value[1:]
    pre_value = value[:-1]
    # 흑자지속
    cond1 = (cur_value > 0) & (pre_value > 0)
    cond1 = np.insert(cond1, 0, np.nan)
    # 적자지속
    cond2 = (cur_value <= 0) & (pre_value <= 0)
    cond2 = np.insert(cond2, 0, np.nan)
    # 흑자전환
    cond3 = (cur_value > 0) & (pre_value <= 0)
    cond3 = np.insert(cond3, 0, np.nan)
    # 적자전환
    cond4 = (cur_value <= 0) & (pre_value > 0)
    cond4 = np.insert(cond4, 0, np.nan)
    
    # 조건에 따른 변환
    data.loc[cond1, col + "_상태"] = "흑자지속"
    data.loc[cond2, col + "_상태"] = "적자지속"
    data.loc[cond3, col + "_상태"] = "흑자전환"
    data.loc[cond4, col + "_상태"] = "적자전환"

In [77]:
add_state(data, "매출액")
add_state(data, "영업이익")
add_state(data, "당기순이익")

In [78]:
data['ROA'] = (data['당기순이익'] / data['자산총계']) * 100

In [79]:
average_equity = data['자본총계'].rolling(2).mean() # 평균 자기 자본
data['ROE'] = (data['당기순이익'] / average_equity) * 100
data.loc[data['연도'] == 2013, 'ROE'] = np.nan

In [80]:
data.to_csv("../../데이터/주요재무지표_수정.csv", index = False, encoding = "euc-kr")

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv("../../데이터/주요재무지표_수정.csv", encoding='euc-kr')

In [3]:
display(data.head())

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율,매출액증가율,영업이익증가율,당기순이익증가율,매출액_상태,영업이익_상태,당기순이익_상태,ROA,ROE
0,3S,2013,6.766058e+10,1.988388e+10,4.777671e+10,3.223645e+10,1.859291e+09,7.011469e+08,41.618347,NaN,NaN,NaN,적자전환,적자전환,적자전환,1.036271,NaN
1,3S,2014,7.131201e+10,2.831919e+10,4.299283e+10,2.886299e+10,-2.540740e+09,-4.760633e+09,65.869563,-10.464742,-236.650986,-778.978003,흑자지속,적자전환,적자전환,-6.675781,-10.489496
2,3S,2015,6.735997e+10,2.942016e+10,3.793981e+10,2.321054e+10,-4.157989e+09,-5.599033e+09,77.544311,-19.583711,63.652721,17.611090,흑자지속,적자지속,적자지속,-8.312107,-13.836280
3,3S,2016,5.774639e+10,2.314263e+10,3.460376e+10,2.374242e+10,1.176536e+09,-3.403438e+09,66.878947,2.291545,-128.295790,-39.213827,흑자지속,흑자전환,적자지속,-5.893767,-9.383155
4,3S,2017,5.370690e+10,2.452724e+10,2.917967e+10,2.648629e+10,-2.421225e+09,-8.803810e+09,84.055918,11.556809,-305.792685,158.674040,흑자지속,적자전환,적자지속,-16.392325,-27.605322
